In [15]:
import pandas as pd
import numpy as np

In [16]:
DATASET_PATH = "../data/raw/"

In [17]:
LABELS = [
    "Walking",
    "Upstairs",
    "Downstairs",
    "Sitting",
    "Standing",
    "Laying"
]

## Load Dataset

In [18]:
def load_data(X_path, y_path):
    with open(X_path, 'r') as file:
        X_ = np.array([np.array(serie, dtype=np.float32) for serie in [row.replace('  ', ' ').strip().split(' ') for row in file]])
    with open(y_path, 'r') as file:
        y_ = np.array([elem for elem in [row.replace('  ', ' ').strip().split(' ') for row in file]],dtype=np.int32).T[0]
    return X_, y_ - 1

In [19]:
X_train, y_train = load_data(DATASET_PATH+"train/X_train.txt", DATASET_PATH+"train/y_train.txt")
X_test, y_test = load_data(DATASET_PATH+"test/X_test.txt", DATASET_PATH+"test/y_test.txt")

## Evaluate Models

In [20]:
from sklearn.model_selection import StratifiedKFold

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC

In [21]:
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

In [22]:
# Random Forests parameters
rf_params = {
    'n_jobs': -1,
    'n_estimators': 500,
     'warm_start': True, 
     #'max_features': 0.2,
    'max_depth': 6,
    'min_samples_leaf': 2,
    'max_features' : 'sqrt',
    'verbose': 0
}

# AdaBoost parameters
ada_params = {
    'n_estimators': 500,
    'learning_rate' : 0.75
}

# Support Vector Machines parameters 
svc_params = {
    'kernel' : 'linear',
    'C' : 0.025
}

In [23]:
models = {
    "Random Forests": RandomForestClassifier(**rf_params),
    "AdaBoost": AdaBoostClassifier(**ada_params),
    "Support Vector Machines": SVC(**svc_params)
}

In [24]:
n_models = len(models)
n_test_samples, _ = X_test.shape

In [26]:
y_pred = np.zeros((n_test_samples, n_models), dtype=np.int32)
for idx, (name, model) in enumerate(models.items()):
    model.fit(X_train, y_train)
    y_pred[:, idx] = model.predict(X_test)

C:\tools\scoop\apps\miniconda3\current\lib\site-packages\sklearn\ensemble\_forest.py:369: UserWarning: Warm-start fitting without increasing n_estimators does not fit new trees.
  warn("Warm-start fitting without increasing n_estimators does not "


In [27]:
y_pred.shape

(2947, 3)

In [28]:
with open('machine-learning.npy', 'wb') as f:
    np.save(f, y_pred)